In [1]:
import numpy as np
import cv2 as cv
import cv2
import glob
import depthai  # depthai - access the camera and its data packets
import blobconverter  # blobconverter - compile and download MyriadX neural network blobs
import time

In [2]:
# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disparity = False
# Better accuracy for longer distance, fractional disparity 32-levels:
subpixel = False
# Better handling for occlusions:
lr_check = True


pipeline = depthai.Pipeline()
cam_rgb = pipeline.create(depthai.node.ColorCamera)
cam_rgb.setPreviewSize(640,480)
cam_rgb.setInterleaved(False)

cam_mono_right = pipeline.create(depthai.node.MonoCamera)
cam_mono_right.setBoardSocket(depthai.CameraBoardSocket.RIGHT)
cam_mono_right.setResolution(depthai.MonoCameraProperties.SensorResolution.THE_480_P)

cam_mono_left = pipeline.create(depthai.node.MonoCamera)
cam_mono_left.setBoardSocket(depthai.CameraBoardSocket.LEFT)
cam_mono_left.setResolution(depthai.MonoCameraProperties.SensorResolution.THE_480_P)

depth = pipeline.create(depthai.node.StereoDepth)
xout = pipeline.create(depthai.node.XLinkOut)
xout_rgb = pipeline.create(depthai.node.XLinkOut)

xout_rgb.setStreamName("rgb")
xout.setStreamName("disparity")

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way)
depth.setDefaultProfilePreset(depthai.node.StereoDepth.PresetMode.HIGH_DENSITY)
# Options: MEDIAN_OFF, KERNEL_3x3, KERNEL_5x5, KERNEL_7x7 (default)
depth.initialConfig.setMedianFilter(depthai.MedianFilter.KERNEL_7x7)
depth.setLeftRightCheck(lr_check)
depth.setExtendedDisparity(extended_disparity)
depth.setSubpixel(subpixel)


# Linking
cam_rgb.preview.link(xout_rgb.input)
cam_mono_left.out.link(depth.left)
cam_mono_right.out.link(depth.right)
depth.disparity.link(xout.input)

In [3]:
prev_frame_time = 0
new_frame_time = 0

with depthai.Device(pipeline) as device:

    # Output queue will be used to get the disparity frames from the outputs defined above
    q_depth = device.getOutputQueue(name="disparity", maxSize=1, blocking=False)
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    fps_captures = list()
    while True:
        inDisparity = q_depth.get()  # blocking call, will wait until a new data has arrived
        frame = inDisparity.getFrame()
        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)

        frame = cv2.applyColorMap(frame, cv2.COLORMAP_JET)
        cv2.imshow("disparity_color", frame)

        in_rgb = q_rgb.tryGet()

        if in_rgb is not None:
            frame = in_rgb.getCvFrame()
            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            fps_captures.append(fps)
            prev_frame_time = new_frame_time
            fps = str(int(fps))
            cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
            cv2.imshow("rgb", frame)
            

        if cv2.waitKey(1) == ord('q'):
            break
cv2.destroyAllWindows()

In [4]:
np_fps = np.array(fps_captures)
np_fps.sort()
print("avg fps: ",int(np_fps.mean()))
print("median fps: ",int(np_fps[int(len(np_fps)/2)]))

avg fps:  33
median fps:  27
